### Component: Explorative Data Analysis

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')



df = pd.read_csv('../data/ds_salaries.csv')

### Step 6. Select
Dropping specific columns from the DataFrame
- **We keep only the columns we need.**

In [2]:
df.drop(df[['salary','salary_currency']], axis = 1, inplace = True)
df

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,US,100,US,L
3752,2020,EN,FT,Data Scientist,105000,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,100000,US,100,US,L


### Step 7. Identify
- **We replace the employment_type column with the full name of the employment type.**
- **We also check the value counts of the employment_type column to see how many of each type of employment there are.**

In [3]:

employment_type = 'employment_type'
df[employment_type] = df[employment_type].replace('FT','Full-Time')
df[employment_type] = df[employment_type].replace('CT','Contract')
df[employment_type] = df[employment_type].replace('PT','Part-Time')
df[employment_type] = df[employment_type].replace('FL','Freelance')
df[employment_type].value_counts()

employment_type
Full-Time    3718
Part-Time      17
Contract       10
Freelance      10
Name: count, dtype: int64

- **We replace the experience level with full names**

In [4]:
experience_level = 'experience_level'
df[experience_level] = df[experience_level].replace('EN','Entry-level/Junior')
df[experience_level] = df[experience_level].replace('MI','Mid-level/Intermediate')
df[experience_level] = df[experience_level].replace('SE','Senior-level/Expert')
df[experience_level] = df[experience_level].replace('EX','Executive-level/Director')
df[experience_level].value_counts()

experience_level
Senior-level/Expert         2516
Mid-level/Intermediate       805
Entry-level/Junior           320
Executive-level/Director     114
Name: count, dtype: int64

In [5]:
print(df.shape)
df.head()

(3755, 9)


,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,Senior-level/Expert,Full-Time,Principal Data Scientist,85847,ES,100,ES,L
1,2023,Mid-level/Intermediate,Contract,ML Engineer,30000,US,100,US,S
2,2023,Mid-level/Intermediate,Contract,ML Engineer,25500,US,100,US,S
3,2023,Senior-level/Expert,Full-Time,Data Scientist,175000,CA,100,CA,M
4,2023,Senior-level/Expert,Full-Time,Data Scientist,120000,CA,100,CA,M


### Step 8. Check
We check for null values
- **We calculate the mean of the salaries for each job title and then we replace low occurrence job titles.**

In [6]:
df.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [7]:
print(df['job_title'].value_counts())

job_title
Data Engineer                          1040
Data Scientist                          840
Data Analyst                            612
Machine Learning Engineer               289
Analytics Engineer                      103
                                       ... 
Principal Machine Learning Engineer       1
Azure Data Engineer                       1
Manager Data Management                   1
Marketing Data Engineer                   1
Finance Data Analyst                      1
Name: count, Length: 93, dtype: int64


In [8]:
print(df['job_title'].value_counts().head(11))
print("Mean of job titles:",df['job_title'].value_counts().mean())
# show job titles with less than 34 occurences
print("Job titles with less than 34:",len(df['job_title'].value_counts()[df['job_title'].value_counts() < 34]))
#print all job titles with more than 34 occurences
print("Job titles with more than 34:",len(df['job_title'].value_counts()[df['job_title'].value_counts() >= 34]))


job_title
Data Engineer                1040
Data Scientist                840
Data Analyst                  612
Machine Learning Engineer     289
Analytics Engineer            103
Data Architect                101
Research Scientist             82
Data Science Manager           58
Applied Scientist              58
Research Engineer              37
ML Engineer                    34
Name: count, dtype: int64
Mean of job titles: 40.376344086021504
Job titles with less than 34: 82
Job titles with more than 34: 11


We will only keep job titles with more than 34 occurrences and use chatgpt to generate job titles mapping

In [9]:
job_title_mapping = {
    # Data Engineer 
    'Big Data Engineer': 'Data Engineer',
    'Data Infrastructure Engineer': 'Data Engineer',
    'Lead Data Engineer': 'Data Engineer',
    'Cloud Database Engineer': 'Data Engineer',
    'ETL Developer': 'Data Engineer',
    'BI Developer': 'Data Engineer',
    'Data Operations Engineer': 'Data Engineer',
    'Cloud Data Engineer': 'Data Engineer',
    'Azure Data Engineer': 'Data Engineer',
    'Data DevOps Engineer': 'Data Engineer',
    'BI Data Engineer': 'Data Engineer',

    # Data Scientist 
    'Machine Learning Scientist': 'Data Scientist',
    'Data Science Consultant': 'Data Scientist',
    'AI Scientist': 'Data Scientist',
    'Applied Data Scientist': 'Data Scientist',
    'Principal Data Scientist': 'Data Scientist',
    'Data Scientist Lead': 'Data Scientist',
    'Product Data Scientist': 'Data Scientist',
    'Compliance Data Analyst': 'Data Scientist', 
    'Data Science Tech Lead': 'Data Scientist',
    'Staff Data Scientist': 'Data Scientist',
    'Data Science Engineer': 'Data Scientist',

    # Data Analyst 
    'Data Manager': 'Data Analyst',
    'Data Analytics Manager': 'Data Analyst',
    'Business Data Analyst': 'Data Analyst',
    'Data Specialist': 'Data Analyst',
    'BI Data Analyst': 'Data Analyst',
    'Data Quality Analyst': 'Data Analyst',
    'Product Data Analyst': 'Data Analyst',
    'Financial Data Analyst': 'Data Analyst',
    'Marketing Data Analyst': 'Data Analyst',
    'Data Modeler': 'Data Analyst',
    'Principal Data Analyst': 'Data Analyst',
    'Insight Analyst': 'Data Analyst',
    'Data Analytics Specialist': 'Data Analyst',
    'Manager Data Management': 'Data Analyst',
    'Finance Data Analyst': 'Data Analyst',
    'BI Analyst': 'Data Analyst',
    'Staff Data Analyst': 'Data Analyst',

    # Machine Learning Engineer 
    'Machine Learning Infrastructure Engineer': 'Machine Learning Engineer',
    'Machine Learning Software Engineer': 'Machine Learning Engineer',
    'Machine Learning Developer': 'Machine Learning Engineer',
    'Deep Learning Engineer': 'Machine Learning Engineer',
    'Machine Learning Research Engineer': 'Machine Learning Engineer',
    'Lead Machine Learning Engineer': 'Machine Learning Engineer',
    'Applied Machine Learning Engineer': 'Machine Learning Engineer',
    'Principal Machine Learning Engineer': 'Machine Learning Engineer',

    # Analytics Engineer 
    'Data Analytics Engineer': 'Analytics Engineer',
    'Lead Data Analyst': 'Analytics Engineer', 
    'Data Analytics Consultant': 'Analytics Engineer',
    'Data Analytics Lead': 'Analytics Engineer',

    # Data Architect 
    'Big Data Architect': 'Data Architect',
    'Principal Data Architect': 'Data Architect',
    'Cloud Data Architect': 'Data Architect',

    # Research Scientist 
    'Machine Learning Researcher': 'Research Scientist',
    '3D Computer Vision Researcher': 'Research Scientist',
    'Deep Learning Researcher': 'Research Scientist',

    # Data Science Manager 
    'Director of Data Science': 'Data Science Manager',
    'Head of Data': 'Data Science Manager',
    'Head of Data Science': 'Data Science Manager',
    'Machine Learning Manager': 'Data Science Manager',
    'Head of Machine Learning': 'Data Science Manager',
    'Lead Data Scientist': 'Data Science Manager',
    'Data Science Lead': 'Data Science Manager',
    'Data Lead': 'Data Science Manager', 

    # Applied Scientist 
    'Applied Machine Learning Scientist': 'Applied Scientist',
    'Computer Vision Engineer': 'Applied Scientist',
    'AI Developer': 'Applied Scientist',
    'NLP Engineer': 'Applied Scientist',
    'Computer Vision Software Engineer': 'Applied Scientist',
    'Autonomous Vehicle Technician': 'Applied Scientist',

    # Research Engineer 
    'MLOps Engineer': 'Research Engineer',
    'Business Intelligence Engineer': 'Research Engineer',
    'Data Operations Analyst': 'Research Engineer',
    'Cloud Data Engineer': 'Research Engineer',
    'AI Programmer': 'Research Engineer',
    'Power BI Developer': 'Research Engineer',
    'Data Management Specialist': 'Research Engineer',
    'Marketing Data Engineer': 'Research Engineer',
    'Data Strategist': 'Research Engineer',
    'Principal Data Engineer': 'Data Engineer',
    'Software Data Engineer': 'Data Engineer',
    'ETL Engineer': 'Data Engineer'

}


In [10]:
df['job_title'] = df['job_title'].replace(job_title_mapping)

In [11]:
print(df['job_title'].value_counts())

job_title
Data Engineer                1110
Data Scientist                935
Data Analyst                  744
Machine Learning Engineer     333
Analytics Engineer            118
Applied Scientist             113
Data Science Manager          111
Data Architect                105
Research Scientist             93
Research Engineer              59
ML Engineer                    34
Name: count, dtype: int64


### Step 9. Structure
- **We will save the cleaned data to a new csv file**

In [12]:
# save csv file
df.to_csv('../data/ds_salaries_cleaned.csv', index = False)

In [13]:
top11_job_titles = df['job_title'].value_counts()[:11]
fig = px.bar(y = top11_job_titles.values, x = top11_job_titles.index, 
            text = top11_job_titles.values, title = 'Top 11 Job Designations')
fig.update_layout(xaxis_title = "Job Designations", yaxis_title = "Count")
fig.show()